### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url,timeout=100)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []

for page_num in range(1, 400):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/lao-dong-tien-luong/t1/11/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))
    

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_laodong_tienthuong = pd.DataFrame(articles_data)
df_laodong_tienthuong.head(5)

,Name,Time,Question,Situation,Answer
0,Nguyễn Văn Nam,08:05 12/04/2025,Thời gian làm hợp đồng có tính hưởng phụ cấp l...,Tôi là giáo viên hợp đồng tại trường THCS thuộ...,Bộ Nội vụ trả lời vấn đề này như sau:Phụ cấp c...
1,Nguyễn Tấn Sửu,08:43 09/04/2025,Các khoản trợ cấp nghỉ hưu trước tuổi tính the...,Điểm c Khoản 1 Điều 5 Nghị định 29/2023/NĐ-CP ...,Bộ Nội vụ trả lời vấn đề này như sau:Khoản 1 Đ...
2,Bùi Thị Nga,08:05 08/04/2025,Cần mấy sáng kiến để xét tặng Bằng khen của tỉnh?,"Tôi có 1 sáng kiến cấp cơ sở, xét khen thưởng ...",Bộ Nội vụ trả lời vấn đề này như sau:Theo quy ...
3,Lê Quang Huy,07:05 08/04/2025,Điều kiện hưởng chính sách nghỉ việc sau sáp n...,"Trước tháng 8/2024, tôi làm hiệu trưởng trường...",Bộ Nội vụ trả lời vấn đề này như sau:Ngày 15/3...
4,Nguyễn Thị Hoa,07:05 07/04/2025,Làm hợp đồng lao động có được bổ nhiệm ngạch?,Tôi đang là nhân viên tại cơ quan Đảng cấp huy...,Bộ Nội vụ trả lời vấn đề này như sau:Trường hợ...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../Source_Fine-Tuning/data/laodong_tienthuong_data.csv'
df_laodong_tienthuong.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../Source_Fine-Tuning/data/laodong_tienthuong_data.csv


In [6]:
df_laodong_tienthuong=pd.read_csv("../Source_Fine-Tuning/data/laodong_tienthuong_data.csv",encoding='utf-8')

In [8]:
df_laodong_tienthuong.shape

(3107, 6)